In [2]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
from os.path import exists

In [3]:
folder = './cv2019_data/calib/'
files = [f'{folder}{f}' for f in os.listdir(folder)]

cameraParameters = './res/params.npz'
LOAD_PARAMETERS = False # if true load parameters from file (if it exists) else computes params from the images

# Finding camera parameters

In [4]:
if LOAD_PARAMETERS and exists(cameraParameters):
    params = np.load(cameraParameters)
    mtx = params['mtx']
    dist = params['dist']
    newcameramtx = params['newcameramtx']
    roi = params['roi']

    print('Camera parameters loaded from file')
else:
    objp = np.zeros((5*7,3), np.float32)
    objp[:,:2] = np.mgrid[0:7,0:5].T.reshape(-1,2)

    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    objpoints = []
    imgpoints = []

    for c, f in enumerate(files):
        img = cv.imread(f)
        gImg = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

        # cv.imshow(f, img)
        # cv.waitKey(0) # waits until a key is pressed
        # cv.destroyAllWindows() # destroys the window showing

        ret, corners = cv.findChessboardCorners(gImg, (7,5), None)

        if ret == True:
            objpoints.append(objp)

            corners2 = cv.cornerSubPix(gImg, corners, (11,11), (-1, -1), criteria)
            imgpoints.append(corners)

            img = cv.drawChessboardCorners(img, (7,5), corners2, ret)
            
            # plt.imshow(img)
            # plt.show()

    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gImg.shape[::-1], None, None)

    h,  w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

    np.savez(cameraParameters, mtx = mtx, dist = dist, newcameramtx = newcameramtx, roi = roi)




## Undistort image

In [5]:
def undistort(img, useRoi=True, plotBeforeAndAfter=False):
    dst = cv.undistort(img, mtx, dist, None, newcameramtx)

    if useRoi:
        x, y, w, h = roi
        dst = dst[y:y+h, x:x+w] # removes black part of the image

    # Plot before and after
    if plotBeforeAndAfter:
        cv.imwrite('./res/calibresult.png', dst)

        fig = plt.figure(figsize=(10, 5))

        fig.add_subplot(1, 2, 1)
        plt.imshow(img)
        plt.axis('off')
        plt.title('Before')

        fig.add_subplot(1, 2, 2)
        plt.imshow(dst)
        plt.axis('off')
        plt.title('After')

    return dst


## Compute Homography

Notes:
- **findContours**: gives a connected contour, Canny instead gives just edges lines that are not necessarily connected
- **approxPolyDP**: It approximates a contour shape to another shape with less number of vertices depending upon the precision we specify (it is an implementation of Douglas-Peucker algorithm.). The second argument is called epsilon, which is maximum distance from contour to approximated contour.
- **arcLength**: contour perimeter, second argument specify wether shape is a closed contour (True) or just a curve (False)
- **rectangle**: the image of the rectangle is 23x13cm, 1cm thick 

In [46]:

def sortRectangles(rectangles):
    pl = []
    # Match The rectangles by hierarchy
    for _, contour, h in rectangles: # Inefficient nested loop, but it's only 4 elements so it's ok
        if h[2] == -1:
            for i2, contour2, _ in rectangles:
                if i2 == h[3]:
                    pl.append([contour, contour2])

    # Reshape the planes array and find the lowest y value
    plane1 = np.concatenate((pl[0][0], pl[0][1]), axis=0)
    plane1 = np.reshape(plane1, (8,2))
    minX1 = np.min(plane1[:,1]) # min y axis

    plane2 = np.concatenate((pl[1][0], pl[1][1]), axis=0)
    plane2 = np.reshape(plane2, (8,2))
    minX2 = np.min(plane2[:,1]) # min y axis

    # return Table - Wall based on the lowest y value
    return (plane2, plane1) if minX1 < minX2 else (plane1, plane2)


def findRectangles(frame, preciseCorners=True):
    frame = undistort(frame)
    gFrame = cv.cvtColor(frame, cv.COLOR_RGB2GRAY)
    r, tFrame = cv.threshold(gFrame, 15, 255, cv.THRESH_BINARY)
    _, contours, hierarchy = cv.findContours(tFrame,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)
    colorG = 255
    colorR = 0

    rectangles = []
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    for i, (contour, h) in enumerate(zip(contours, hierarchy[0])):
        perimeter = cv.arcLength(contour,True)
        epsilon = 0.005*perimeter
        approx = cv.approxPolyDP(contour, epsilon, True)
        if len(approx) == 4 and perimeter > 20:
            if preciseCorners:
                corners = cv.cornerSubPix(tFrame, np.float32(approx), (5,5), (-1, -1), criteria)
                corners = np.int64(corners)
                rectangles.append((i, corners, h))
                frame = cv.drawContours(frame, [corners], -1, (0, colorG, colorR))
            else:
                rectangles.append((i, approx, h))
                frame = cv.drawContours(frame, [approx], -1, (0, colorG, colorR))
            colorG -= 75
            colorR += 75
          

    # plt.imshow(frame)
    # plt.show()
    # cv.imshow('rectangles', frame)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    outName = './res/planes_precise.png' if preciseCorners else './res/planes_approx.png'
    #cv.imwrite(outName, frame)

    table, wall = sortRectangles(rectangles)  

    return table, wall


vid = cv.VideoCapture('./cv2019_data/cup1.mp4')
ret, frame = vid.read()

if ret:
    table, wall = findRectangles(frame, preciseCorners=True)

## Homography
Returns a 3x3 matrix, R1 R2 and T columns
the rows are x, y and constant

to calculate the point in the ideal plane, the formula of x is

https://docs.opencv.org/3.4/d9/d0c/group__calib3d.html#gafd3ef89257e27d5235f4467cbb1b6a63

R3 is the crossproduct between R1 and R2 and allows to understand the distance in X from the image plane
T represent the point in the ideal plane with coordinates of image plane

R3 se ha tutti gli elementi negativi vuol dire che il piano va di fronte a sinistra della camera
Se ha neg neg pos, vuol dire che fa up left far away from the camera
se ha neg pos neg, va down left to the camera
se ha pos neg neg, va sopra a destra verso la camera

In [7]:
# TODO revise the math in slide 11
def findPlanes(rectangle, mtxInv):
    originalRectangle = np.array([[0,0], [0,13], [25, 13], [25, 0], [1,1], [1, 12], [24, 12], [24, 1]])
    #originalRectangle = np.array([[0,13], [25, 13], [25, 0], [0, 0], [1,1], [1, 12], [24, 12], [24, 1]])

    # 3x3 matrix
    homography, _ = cv.findHomography(originalRectangle, rectangle)

    H = np.dot(mtxInv, homography)
    r1 = H[:, 0]
    r2 = H[:, 1]
    t = H[:, 2]

    s1 = np.linalg.norm(r1)
    s2 = np.linalg.norm(r2)
    s = (s1 + s2)/2.0
    
    r1 = r1/s1
    r2 = r2/s2
    r3 = np.cross(r1, r2)
    r3 = r3 / np.linalg.norm(r3)
    t = t/s
    
    print('plane normal: ', r3)
    print('plane point: ', t)
    
    return t, r3, homography
   


invMtx = np.linalg.inv(mtx)
tableP = findPlanes(table, invMtx)
wallP = findPlanes(wall, invMtx)



plane normal:  [ 0.30509558  0.95213912 -0.01864916]
plane point:  [-0.00303203  0.00105186  0.01200025]
plane normal:  [-0.47864474 -0.83868687 -0.25981444]
plane point:  [ 0.00799469 -0.08210011  0.25116785]


In [41]:
def rectBbox(rect):
    print(rect)
    minX = np.min(rect[:, 0])
    minY = np.min(rect[:, 1])
    maxX = np.max(rect[:, 0])
    maxY = np.max(rect[:, 1])
    print(minX, maxX, minY, maxY)

def maskLaser(frame, table, wall):
    f = cv.convertScaleAbs(frame, alpha=1.4, beta=50)
    f = cv.GaussianBlur(f, (3,3), 0)
    f = cv.cvtColor(f, cv.COLOR_BGR2HLS)

    cv.imshow('converted', f)

    lower_red = np.array([140, 50, 50])
    upper_red = np.array([225, 255, 255])

    mask = cv.inRange(f, lower_red, upper_red)

    f = cv.bitwise_and(f, f, mask = mask)
 
    #cv.pointPolygonTest(table, (x,y), False) 

    # plt.imshow(frame)
    # plt.show()
    # plt.imshow(f)

    cv.imshow('original', frame)
    cv.imshow('laser', f)
    cv.waitKey(1)

vid = cv.VideoCapture('./cv2019_data/cup1.mp4')
for i in range(300):
    ret, frame = vid.read()
    maskLaser(frame, None, None)
cv.destroyAllWindows()
rectBbox(table)

[[355 628]
 [795 638]
 [753 508]
 [401 502]
 [334 638]
 [389 497]
 [764 502]
 [815 648]]
334 815 497 648
